
In this section, calcium fluxes in a compartmentalized cell were modelled. Calcium can be present in the cytosol ($Ca_{cyt}$), bound to calcium binding proteins residing in the cytosol ($CaPr$), the Endoplasmic Reticulum (ER) ($Ca_{ER}$) as well as the mitochondria ($Ca_m$). Fluxes between these compartments occur, leading to a an exchange of calcium. The overall setup of the system is shown in Figure X.
![title](Scheme.png)
Figure X: Calcium fluxes within a cell.

Different formulas are used to describe this system.
The total calcium concentration ($Ca_{tot}$) can be defined as:

$Ca_{tot} = Ca_{cyt} + \frac{\rho_{ER}}{\beta_{ER}}+Ca_{ER}+\frac{\rho_{m}}{\beta_{m}}+Ca_{m}+CaPr$

The total amount of the calcium binding protein ($Pr_{tot}$) in it's bound ($CaPr$) or unbound ($Pr$) form is defined as:

$Pr_{tot} = Pr + CaPr$

To describe the rate of change of calcium concentrations in the different compartments the following formulas can be used:

$\frac{dCa_{cyt}}{dt} = J_{ch} + J_{leak} - J_{pump} + J_{out} - J_{in} + k_- CaPr - k_+ Ca_{cyt}Pr$

$\frac{dCa_{ER}}{dt} = \frac{\beta_{ER}}{\rho_{ER}}(J_{pump} - J_{ch} - J_{leak})$

$\frac{dCa_{m}}{dt} = \frac{\beta_{m}}{\rho_{m}}(J_{in} - J_{out})$

Where the fluxes ($J$) are defined as:

$J_{pump} = k_{pump}Ca_{cyt}$

$J_{ch} = k_{ch}\frac{Ca_{cyt}^2}{K_1^2+Ca_{cyt}^2}(Ca_{ER}-Ca_{cyt})$

$J_{leak} = k_{leak}(Ca_{ER} - Ca_{cyt})$

$J_{in} = k_{in}\frac{Ca_{cyt}^8}{K_2^8+Ca_{cyt}^8}$

$J_{out} = (k_{out}\frac{Ca_{cyt}^2}{K_3^2+Ca_{cyt}^2}+k_m)Ca_{m}$



$J_{pump}$ is the outflux from the cytosol into the ER via ion pumps. It depends on the rate of the ion pump (k_{pump}) as well as the current concentration of cytosolic calcium. Higher calcium concentration, as well as a more active pump lead to a higher $J_{pump}$.

$J_{ch}$ defines the flux from the ER into the cystosol via calcium channels. $k_{ch}\frac{Ca_{cyt}^2}{K_1^2+Ca_{cyt}^2}$ is the maximal rate of the calcium channel ($k_{ch}$) scaled by a Hill term that takes into account the affinity of the channel for calcium ($K_1$) and the current calcium concentration in the cytosol, together with a Hill coefficient of 2. The direction of the flux, as well as the extend is further modified by the difference in $Ca_{ER}$ and $Ca_{cyt}$. Since transport through the channels is passive, it will seak to establish equilibrium, by producing a flux from the compartment with the higher concentration to the one with the lower one. A bigger gradient between the two of them lead to a stronger flux.

$J_{leak}$ describes the leakage of calcium ions through the membrane of the ER into the cytosol. It depends on the maximum rate at which ions can pass trough the membrane ($k_{leak}$) and analogically to $J_{ch}$ the gradient between Ca_{ER}$ and $Ca_{cyt}$.

$J_{in}$ is defined as the influx of $Ca^{2+}$ ions from the cytosol to the mitochondria. It is decribes by a the maximal inflow rate $k_{in}$ and a Hill term ($\frac{Ca_{cyt}^8}{K_2^8+Ca_{cyt}^8}$), that takes into account the max, the the affinity parameter $K_{2}, as well as cytosolic calcium concentrations, and has a hill coefficient of 8.

Lastly the formula of $J_{out}$ contains $k_m$ as a term to describe leakage from the mitochondria to the cytosol, as well as $k_{out}\frac{Ca_{cyt}^2}{K_3^2+Ca_{cyt}^2}$ as the product of maximal non-leakage outflow and a Hill term with the Hill coefficient 2. The sum of these two terms is scaled by the current concentration of mitochondrial calcium.

The fact that both $J_{in}$ and $J_{out}$ contain Hill terms, indicates that these fluxes are partly mediated by proteins, like ion channels or ion pumps, although these are not indicated in the scheme. These mitochondrial membrane proteins, as well as the ion channels situated in the ER membrane appear to consist of several subunits, that display cooperativity. The Hill coefficients indicate that the ER ion channel, and the proteins mediating mitochondrial outflux might consist of two cooperating units, while the proteins mediating the influx of calcium into the mitochondria might consist of eight such subunits.

In [1]:
#Import statements
# Basic Imports
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random
import copy
from scipy.integrate import odeint
from ipywidgets import interact, interactive, fixed
from IPython.display import clear_output, display, HTML

# Declare parameters
#---------------------------------------
#Total concentrations (micro Moles)
Ca_tot =  90 
Pr_tot = 120

# Geometric Parameters

rho_ER = 0.01
rho_m = 0.01
beta_ER = 0.0025
beta_m = 0.0025

#Kinetics parameters

k_ch = 4100 #s^-1
k_pump = 20 #s^-1
k_leak = 0.01 #s^-1
k_in = 300 #(micro Moles s^-1)
k_out = 125 #s^-1
k_m = 0.00625 #s^-1
k_plus = 0.1 #(micro Moles)
k_minus = 0.01 #s^-1
K1 = 5 #(micro Moles)
K2 = 0.8 #(micro Moles)
K3 = 5 #(micro Moles)

#Initial concentrations (micro Moles)

Ca_cyt_0 = 0.3 
Ca_ER_0 = 0.2
Ca_m_0 = 1

# Define the equations
# Do we need to solve Ca_tot for CaPr in otder to get Pr from the next equation recursivly?  Or are we supposed to use a package for this?

# Define the differential equations
# Are we supposed to do this discretely or continuosly?  How are we supposed to solve these?


def system_ODE():

# Define the flux equations

def fluxPump(k_pump,Ca_cyt):
    return k_pump*Ca_cyt

def fluxCh(k_ch, Ca_cyt,Ca_ER,K1):
    return k_ch*((Ca_cyt**2)/(K1**2 + Ca_cyt**2))*(Ca_ER - Ca_cyt)

def fluxIn(k_in, K2, Ca_cyt):
    return k_in*(Ca_cyt**8/(K2**8 + Ca_cyt**8))

def fluxOut(k_out, K3, Ca_cyt, km, Ca_m):
    return Ca_m*(k_out*(Ca_cyt**2 / (K3**2 + Ca_cyt**2))+km)

IndentationError: expected an indented block (<ipython-input-1-00c446181b24>, line 58)